# Define Weight Path

In [ ]:
kyu_weight_path = './Kyu_Final_Weight.pth'
dan_weight_path = './Dan_Final_Weight.pth'

# Import

In [ ]:
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchsummary import summary
from thop import profile

# Prepare Input

In [ ]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
dirR = [-1, -1, -1, 0, 0, 1, 1, 1, -2, -2, -2, -2, -2, -1, -1, 0, 0, 1, 1, 2, 2 ,2 ,2 ,2, -3, -3, -3, -3, -3, -3, -3
       , -2, -2, -1, -1, 0, 0, 1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 3]
dirC = [-1, 0, 1, -1, 1, -1, 0, 1, -2, -1, 0, 1, 2, -2, 2, -2, 2, -2, 2, -2, -1, 0, 1, 2, -3, -2, -1, 0, 1, 2, 3, -3
       , 3, -3, 3, -3, 3, -3, 3, -3, 3, -3, -2, -1, 0, 1, 2, 3]


def prepare_input(moves, color):
    x = np.zeros((19,19,21))
    
    lstn = 5
    if(len(moves) < lstn):
        lstn = len(moves)
    
    for i in range(1,lstn+1):
        column = coordinates[moves[-i][2]]
        row = coordinates[moves[-i][3]]
        x[row, column, 21 - i] = 1
        
    
    for move in moves:
        colors = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if colors == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if colors == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
            
    my_piece = x[:,:,0]
    opt_piece = x[:,:,1]
    
    tmp_my_piece = np.empty((19,19))
    tmp_my_piece[::, 0:-1] = my_piece[::, 1:]
    tmp_my_piece[::, [-1]] = 0
    x[:,:,4] += tmp_my_piece
    
    tmp_my_piece = np.empty((19,19))
    tmp_my_piece[::, 1:] = my_piece[::, 0:-1]
    tmp_my_piece[::, [0]] = 0
    x[:,:,4] += tmp_my_piece
    
    tmp_my_piece = np.empty((19,19))
    tmp_my_piece[0:-1, ::] = my_piece[1:, ::]
    tmp_my_piece[[-1], ::] = 0
    x[:,:,4] += tmp_my_piece
    
    tmp_my_piece = np.empty((19,19))
    tmp_my_piece[1:, ::] = my_piece[0:-1, ::]
    tmp_my_piece[[0], ::] = 0
    x[:,:,4] += tmp_my_piece
    
    tmp_opt_piece = np.empty((19,19))
    tmp_opt_piece[::, 0:-1] = opt_piece[::, 1:]
    tmp_opt_piece[::, [-1]] = 0
    x[:,:,9] += tmp_opt_piece
    
    tmp_opt_piece = np.empty((19,19))
    tmp_opt_piece[::, 1:] = opt_piece[::, 0:-1]
    tmp_opt_piece[::, [0]] = 0
    x[:,:,9] += tmp_opt_piece
    
    tmp_opt_piece = np.empty((19,19))
    tmp_opt_piece[0:-1, ::] = opt_piece[1:, ::]
    tmp_opt_piece[[-1], ::] = 0
    x[:,:,9] += tmp_opt_piece
    
    tmp_opt_piece = np.empty((19,19))
    tmp_opt_piece[1:, ::] = opt_piece[0:-1, ::]
    tmp_opt_piece[[0], ::] = 0
    x[:,:,9] += tmp_opt_piece
                        
    x[:,:,3] = np.where(x[:,:,2] == 1, 0, 1)    
        
    x[:,:,5] = np.where(x[:,:,4] == 1, 1, 0)
    x[:,:,6] = np.where(x[:,:,4] == 2, 1, 0)
    x[:,:,7] = np.where(x[:,:,4] == 3, 1, 0)
    x[:,:,8] = np.where(x[:,:,4] == 4, 1, 0)
    x[:,:,4] = np.where(x[:,:,4] == 0, 1, 0)
   
    x[:,:,10] = np.where(x[:,:,9] == 1, 1, 0)
    x[:,:,11] = np.where(x[:,:,9] == 2, 1, 0)
    x[:,:,12] = np.where(x[:,:,9] == 3, 1, 0)
    x[:,:,13] = np.where(x[:,:,9] == 4, 1, 0)
    x[:,:,9] = np.where(x[:,:,9] == 0, 1, 0)
        
    x[:,:,15] = np.where(x[:,:,14] == 0, 1, 0)    
        
    x = np.transpose(x, (2, 0, 1))
    if color == 'W':
        x[[0, 1], :, :] = x[[1, 0], :, :]
        x[[4, 9], :, :] = x[[9, 4], :, :]
        x[[5, 10], :, :] = x[[10, 5], :, :]
        x[[6, 11], :, :] = x[[11, 6], :, :]
        x[[7, 12], :, :] = x[[12, 7], :, :]
        x[[8, 13], :, :] = x[[13, 8], :, :]
    return x
    #x[0, :, :]放要預測的棋子, x[1, :, :]放對手的棋子, x[2, :, :]放有棋子的地方

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    index = column*19+row
    return torch.tensor(index, dtype=torch.long)

def ToImage(game):
    game = np.array(game)
    plt.imshow(game, cmap='gray')
    plt.show()

# Kyu Model

In [ ]:
class InputBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1):
        super(InputBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=5, stride=1, padding=2, bias=False)
    
    def forward(self, x):
        out = self.conv1(x) + self.conv2(x)
        out = F.relu(out)
        return out

class SqueezeExcitationBlock(nn.Module):
    def __init__(self, in_planes, ratio):
        super(SqueezeExcitationBlock, self).__init__()
        self.dense1 = nn.Linear(in_planes, in_planes // ratio, bias=False)
        self.dense2 = nn.Linear(in_planes // ratio, in_planes, bias=False)
        
    def forward(self, x):
        se = F.adaptive_avg_pool2d((x), (1, 1))
        se = se.reshape((se.shape[0], -1))
        se = F.relu(self.dense1(se))
        se = F.sigmoid(self.dense2(se))     
        x = x * se.unsqueeze(dim=-1).unsqueeze(dim=-1)
        return x
    
class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1, ratio=16):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.seblock = SqueezeExcitationBlock(out_planes, ratio)
        self.shortcut = nn.Sequential()
    
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.conv2(out)
        out += self.shortcut(x)
        out = F.relu(out)
        out = self.seblock(out)
        return out
        
class ResNet(nn.Module):
    def __init__(self, in_planes=4, num_blocks=20, num_classes=361, ratio=16):
        super(ResNet, self).__init__()
        self.layer1 = InputBlock(in_planes, out_planes=128, stride=1)
        self.layer2 = self.make_layer(BasicBlock, 128, num_blocks, ratio)
        self.layer3 = nn.Conv2d(128, 1, kernel_size=3, stride=1, padding=1, bias=False)
        self.flatten = nn.Flatten()
    
    def make_layer(self, block, planes, num_blocks, ratio):
        layers = []
        for n in range(num_blocks):
            layers.append(block(planes, planes, 1, ratio))
        return nn.Sequential(*layers)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.flatten(out)
        return out

In [ ]:
kyu_model = ResNet(in_planes=21, num_blocks=20, num_classes=361).to('cuda')

In [ ]:
input = torch.rand(1, 21, 19, 19).cuda()

kyu_model.eval()
out = kyu_model(input)

summary(kyu_model, input_size=(21, 19, 19))
print(f'From input shape: {input.shape} to output shape: {out.shape}')


macs, parm = profile(kyu_model, inputs=(input, ))
print(f'FLOPS: {macs * 2 / 1e9} G, Params: {parm / 1e6} M.')

# Load Kyu Model Weight

In [ ]:
checkpoint = torch.load(kyu_weight_path)
kyu_model.load_state_dict(checkpoint['model_state_dict'], strict=True)

# Dan Model

In [ ]:
class InputBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1):
        super(InputBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=5, stride=1, padding=2, bias=False)
    
    def forward(self, x):
        out = self.conv1(x) + self.conv2(x)
        out = F.relu(out)
        return out

class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.shortcut = nn.Sequential()
    
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.conv2(out)
        out += self.shortcut(x)
        out = F.relu(out)
        return out
        
class ResNet(nn.Module):
    def __init__(self, in_planes=4, num_blocks=20, num_classes=361):
        super(ResNet, self).__init__()
        self.layer1 = InputBlock(in_planes, out_planes=256, stride=1)
        self.layer2 = self.make_layer(BasicBlock, 256, num_blocks)
        self.layer3 = nn.Conv2d(256, 1, kernel_size=3, stride=1, padding=1, bias=False)
        self.flatten = nn.Flatten()
    
    def make_layer(self, block, planes, num_blocks):
        layers = []
        for n in range(num_blocks):
            layers.append(block(planes, planes))
        return nn.Sequential(*layers)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.flatten(out)
        return out

In [ ]:
dan_model = ResNet(in_planes=21, num_blocks=20, num_classes=361).to('cuda')

In [ ]:
input = torch.rand(1, 21, 19, 19).cuda()

dan_model.eval()
out = dan_model(input)

summary(dan_model, input_size=(21, 19, 19))
print(f'From input shape: {input.shape} to output shape: {out.shape}')


macs, parm = profile(dan_model, inputs=(input, ))
print(f'FLOPS: {macs * 2 / 1e9} G, Params: {parm / 1e6} M.')

# Load Dan Model Weight

In [ ]:
checkpoint = torch.load(dan_weight_path)
dan_model.load_state_dict(checkpoint['model_state_dict'], strict=True)

# Make Upload File

In [ ]:
def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    resulting_preds_numbers = [np.flip(np.argsort(prediction)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

chartonumbers = {k:v for k,v in enumerate(chars)}

In [ ]:
df = open('./29_Public Testing Dataset_Public Submission Template_v2/29_Public Testing Dataset_v2/dan_test_public.csv').read().splitlines()
color = [line.strip().split(',')[1] for line in df]
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for idx, game in enumerate(games):
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list, color[idx]))

with open('./upload.csv', 'a+') as f:
    for idx, test_data in enumerate(tqdm(x_testing)):
        test_data = torch.from_numpy(np.array(test_data)).float().to('cuda')
        predictions = dan_model(test_data).to('cpu').detach().numpy()
        prediction_chars = top_5_preds_with_chars(predictions)
        answer_row = games_id[idx] + ',' + ','.join(prediction_chars[0]) + '\n'
        f.write(answer_row)

In [ ]:
df = open('./29_Private Testing Dataset_Public and Private Submission Template_v2/29_Private Testing Dataset_v2/dan_test_private.csv').read().splitlines()
color = [line.strip().split(',')[1] for line in df]
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for idx, game in enumerate(games):
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list, color[idx]))

with open('./upload.csv', 'a+') as f:
    for idx, test_data in enumerate(tqdm(x_testing)):
        
        test_data = torch.from_numpy(np.array(test_data)).float().to('cuda')
        test_data = test_data.reshape(1, test_data.shape[0], 19, 19)
        predictions = dan_model(test_data).to('cpu').detach().numpy()
        prediction_chars = top_5_preds_with_chars(predictions)
        answer_row = games_id[idx] + ',' + ','.join(prediction_chars[0]) + '\n'
        f.write(answer_row)

In [ ]:
df = open('./29_Public Testing Dataset_Public Submission Template_v2/29_Public Testing Dataset_v2/kyu_test_public.csv').read().splitlines()
color = [line.strip().split(',')[1] for line in df]
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for idx, game in enumerate(games):
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list, color[idx]))

with open('./upload.csv', 'a+') as f:
    for idx, test_data in enumerate(tqdm(x_testing)):
        test_data = torch.from_numpy(np.array(test_data)).float().to('cuda')
        test_data = test_data.reshape(1, test_data.shape[0], 19, 19)
        predictions = kyu_model(test_data).to('cpu').detach().numpy()
        prediction_chars = top_5_preds_with_chars(predictions)
        answer_row = games_id[idx] + ',' + ','.join(prediction_chars[0]) + '\n'
        f.write(answer_row)

In [ ]:
df = open('./29_Private Testing Dataset_Public and Private Submission Template_v2/29_Private Testing Dataset_v2/kyu_test_private.csv').read().splitlines()
color = [line.strip().split(',')[1] for line in df]
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for idx, game in enumerate(games):
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list, color[idx]))

with open('./upload.csv', 'a+') as f:
    for idx, test_data in enumerate(tqdm(x_testing)):
        
        test_data = torch.from_numpy(np.array(test_data)).float().to('cuda')
        test_data = test_data.reshape(1, test_data.shape[0], 19, 19)
        predictions = kyu_model(test_data).to('cpu').detach().numpy()
        prediction_chars = top_5_preds_with_chars(predictions)
        answer_row = games_id[idx] + ',' + ','.join(prediction_chars[0]) + '\n'
        f.write(answer_row)